In [1]:
import cv2 as cv
print(cv.__version__)

4.4.0


In [17]:
img_test = cv.imread('C:/Users/Ryu/Desktop/CTSkinSegmentation/CTSkinSegmentation/test.png')
print(type(img_test))
print(img_test.shape)
print(img_test[90:,90:,0])

print()

img_cvtGray = cv.cvtColor(img_test, cv.COLOR_BGR2GRAY)
print(img_cvtGray[90:,90:])
print(img_cvtGray.shape)

<class 'numpy.ndarray'>
(180, 279, 3)
[[ 45  45  45 ... 255 255 255]
 [ 45  45  45 ... 255 255 255]
 [ 45  45  45 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]

[[ 46  46  46 ... 255 255 255]
 [ 46  46  46 ... 255 255 255]
 [ 46  46  46 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
(180, 279)


In [3]:
# 얕은 복사(같은 id) vs 깊은 복사(다른 id)

# 1. 얕은 복사
a = [1,2,3]
b = a
print(id(a), id(b)) # 같은 id (같은 주소값)

b[0] = 5000
print(a, b) # 같은 곳을 가리키므로, b를 바꿔도 a값까지 바뀜!

print()

# 2. 깊은 복사
c = [1,2,3]
d = c.copy()
print(id(c), id(d))

d[0] = 5000
print(c, d)

2900486167688 2900486167688
[5000, 2, 3] [5000, 2, 3]

2900486167880 2900486167624
[1, 2, 3] [5000, 2, 3]


### cv2.equalizeHist(ds.pixel_array)
The function equalizeHist is histogram equalization of images and only implemented for CV_8UC1 type, which is a single channel 8 bit unsigned integral type.

In [28]:
import cv2 as cv
import glob
import pydicom as dicom
import os
import numpy as np
from matplotlib import pyplot, cm

def DcmToMat(ori, shape, files_str):
    # The array is sized based on 'ConstPixelDims'
    ArrayDicom = np.zeros(shape, dtype=np.uint8) # ArrayDicom.shape : (512, 512, 193) ==> uint16! ==> should cvt to uint8!

    # loop through all the DICOM files
    cnt = 0
    for filenameDCM in files_str:
        if cnt == 1 : break
        # read the file
        ds = dicom.read_file(filenameDCM)
        # store the raw image data
        ArrayDicom[:, :, files_str.index(filenameDCM)] = ds.pixel_array
        
        print(ds.pixel_array[256:, 256:]) # ***error*** ==> out of range (1000넘는 값들 있음...!)
        
        print()
        
        dst = cv.equalizeHist(ArrayDicom[:, :, 0])
        print(dst)
        
        cnt += 1
        
    return ArrayDicom[:, :, 0]

def main():
    path = "C:/Users/Ryu/Desktop/180509_SampleData_CT"
    files_str = [file for file in glob.glob("C:/Users/Ryu/Desktop/180509_SampleData_CT/*.dcm")] # imread(dcm) : None
    
    cnt = 0
    for f in files_str:
        if cnt == 1: break
        ori = dicom.read_file(f) # get dicom file / type : <class 'pydicom.dataset.FileDataset'> / size : (512, 512)       
        rows, cols = ori.Rows, ori.Columns
        shape = (rows, cols, 193)
        ori1 = DcmToMat(ori, shape, files_str) # type : np.ndarray ==> cv.imread img도 np.ndarray형!
        
        img_copy = ori1.copy()
        cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/img_copy.png', img_copy) 
        #print(ori1, ori1.shape)
        
        
        ''' 이건 원본대로 잘 보임
        ConstPixelSpacing = (float(ori.PixelSpacing[0]), float(ori.PixelSpacing[1]), float(ori.SliceThickness))
        x = np.arange(0.0, (513)*ConstPixelSpacing[0], ConstPixelSpacing[0])
        y = np.arange(0.0, (513)*ConstPixelSpacing[1], ConstPixelSpacing[1])
        z = np.arange(0.0, (513)*ConstPixelSpacing[2], ConstPixelSpacing[2])
        pyplot.figure(dpi=300)
        pyplot.axes().set_aspect('equal', 'datalim')
        pyplot.set_cmap(pyplot.gray())
        pyplot.pcolormesh(x, y, np.flipud(ori1))
        '''
        
        # 1. Bilateral Filtering (noise filtering)
        
        
        cnt+=1
    
    
main()

[[1020 1018 1018 ... 1069 1044 1056]
 [1008 1031 1047 ... 1074 1049 1066]
 [1021 1041 1051 ... 1056 1034 1050]
 ...
 [ 146  147  152 ...   21   29   18]
 [   7    0    0 ...   19   31   21]
 [   3    3    0 ...   18   26   22]]

[[ 0  0  0 ... 25 74 88]
 [ 0  0  0 ... 50 84 96]
 [ 0  0  0 ... 45 74 79]
 ...
 [ 0  0  0 ... 45 88 30]
 [ 0  0  0 ... 34 96 45]
 [ 0  0  0 ... 30 74 50]]
